In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from src.data import Data
from src.plothelp import *
import time

### Load and pre-process the dataset:


In [3]:
data = Data("img")
data.load_pickle("28x28.pickle ")
X_train, X_test, y_train, y_test = data.train_test_split()

In [4]:
#Convert python lists to np arrays
X_trains = np.asarray(X_train)
X_tests = np.asarray(X_test)
y_trains = np.asarray(y_train)
y_tests = np.asarray(y_test)
print(X_trains.shape, X_tests.shape, y_trains.shape, y_tests.shape)

(4698, 28, 28) (522, 28, 28) (4698,) (522,)


In [5]:
y_oh, y_test_oh= data.one_hot(y_trains, y_tests)

In [6]:
# Split y_train into 54 batches containing 87 img
y_batch = np.array_split(y_oh, 54 )
y_batch = np.asarray(y_batch)
print(y_batch.shape)

(54, 87, 4)


In [7]:
# Split X_test into 54 batches of 87 img
X_train_batch = np.array_split(X_trains, 54 )
X_train_batch = np.asarray(X_train_batch)
X_train_batch = np.expand_dims(X_train_batch, axis=-1)
print(X_train_batch.shape)

(54, 87, 28, 28, 1)


In [8]:
X_tests = np.expand_dims(X_tests, axis=3)
X_tests.shape

(522, 28, 28, 1)

In [9]:
learning_rate = 0.003
batch_size = 87
n_epochs = 40
lr = 0.001

In [10]:
# input X: 28x28 grayscale images
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 4])

In [11]:
l1 = 4  # first convolutional layer output depth
l2 = 8  # second convolutional layer output depth
l3 = 12  # third convolutional layer
l4 = 200  # fully connected layer
num_classes = 4 

In [12]:
w1 = tf.Variable(tf.truncated_normal([5, 5, 1, l1], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
b1 = tf.Variable(tf.random_normal([l1]))

w2 = tf.Variable(tf.truncated_normal([5, 5, l1, l2], stddev=0.1))
b2 = tf.Variable(tf.random_normal([l2]))

w3 = tf.Variable(tf.truncated_normal([4, 4, l2, l3], stddev=0.1))
b3 = tf.Variable(tf.random_normal([l3]))

w4 = tf.Variable(tf.truncated_normal([7 * 7 * l3, l4], stddev=0.1))
b4 = tf.Variable(tf.random_normal([l4]))

w = tf.Variable(tf.truncated_normal([l4, num_classes], stddev=0.1))
b = tf.Variable(tf.random_normal([num_classes]))

In [13]:
# The model
Y1 = tf.nn.relu(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME') + b1)

Y2 = tf.nn.conv2d(Y1, w2, strides=[1, 1, 1, 1], padding='SAME')
Y2 = tf.nn.max_pool(Y2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')  # reduce to 14x14
Y2 = tf.nn.relu( Y2+ b2)

Y3 = tf.nn.conv2d(Y2, w3, strides=[1, 1, 1, 1], padding='SAME')
Y3 = tf.nn.max_pool(Y3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')  # reduce to 7x7
Y3 = tf.nn.relu(Y3+ b3)

# fully connected layer
fc = tf.reshape(Y3, shape=[-1, 7 * 7 * l3])

Y4 = tf.nn.relu(tf.matmul(fc, w4) + b4)
Ylogits = tf.matmul(Y4, w) + b
logits = tf.nn.softmax(Ylogits)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy)

# Calculate the accuracy of the model
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [14]:

init = tf.global_variables_initializer()

In [15]:
with tf.Session() as sess:
    sess.run(init)
    n_batches = 54
    for epochs in range(68):
        acc = 0
        for i in range(n_batches):
            batch_X = X_train_batch[i, :]
            batch_Y = y_batch[i, :]
            _, a = sess.run([train_step, accuracy], {X: batch_X, Y: batch_Y})
            acc=acc+a
        print("Epoch:", '%04d' % (epochs + 1), "accuracy=", "{:.9f}".format(acc/54))

    saver = tf.train.Saver()
    save_path = saver.save(sess, "./models/conv28x28-2.ckpt")
    print("Model saved in file: %s" % save_path)

Epoch: 0001 accuracy= 0.263516390
Epoch: 0002 accuracy= 0.326309071
Epoch: 0003 accuracy= 0.371434653
Epoch: 0004 accuracy= 0.422094507
Epoch: 0005 accuracy= 0.469774370
Epoch: 0006 accuracy= 0.537462751
Epoch: 0007 accuracy= 0.583439759
Epoch: 0008 accuracy= 0.601958280
Epoch: 0009 accuracy= 0.625159641
Epoch: 0010 accuracy= 0.620689657
Epoch: 0011 accuracy= 0.656662408
Epoch: 0012 accuracy= 0.625798213
Epoch: 0013 accuracy= 0.684972330
Epoch: 0014 accuracy= 0.729246489
Epoch: 0015 accuracy= 0.769476373
Epoch: 0016 accuracy= 0.710727969
Epoch: 0017 accuracy= 0.730097912
Epoch: 0018 accuracy= 0.738612178
Epoch: 0019 accuracy= 0.690080880
Epoch: 0020 accuracy= 0.815666236
Epoch: 0021 accuracy= 0.862494678
Epoch: 0022 accuracy= 0.797999146
Epoch: 0023 accuracy= 0.754789272
Epoch: 0024 accuracy= 0.767773521
Epoch: 0025 accuracy= 0.770540657
Epoch: 0026 accuracy= 0.803959131
Epoch: 0027 accuracy= 0.859301824
Epoch: 0028 accuracy= 0.858237545
Epoch: 0029 accuracy= 0.872073220
Epoch: 0030 ac

In [17]:
with tf.Session() as sess:
    sess.run(init)
    saver = tf.train.Saver().restore(sess, "./models/conv28x28-2.ckpt")
    print("Model restored.")
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={X: X_tests, Y: y_test_oh}))

Model restored.
0.381226


The convolutional neural network with 3 convlutional layers and one fully connected layer. It achieved very high accuracy while training on the dataset. However, tested model achieved only 38.12 % accuracy. Hypothesis why training accuracy is so high while test accuracy is low is that the learnt model overfits the training data. The other possibility is that neural network is not deep enought and needs more layers. Adding a dropout for neurons should improve the classifier. The model was saved as conv28x28-2.ckpt